<a href="https://colab.research.google.com/github/cagBRT/Data/blob/main/Imbalanced_Dataset_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In this notebook we look at a real imbalanced dataset, as opposed to synthetic data.**

We compare various logistic regression models using cross validation - leave one out - to find the best model for this dataset.

In [ ]:
# Clone the entire repo.
!git clone -s https://github.com/cagBRT/Data.git cloned-repo
%cd cloned-repo

In [ ]:
import pandas as pd
from pandas import read_csv
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
from pandas.plotting import scatter_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from matplotlib.ticker import FormatStrFormatter
from sklearn import model_selection
from sklearn import metrics

In [ ]:
from numpy import mean
from numpy import isnan
from numpy import asarray
from numpy import polyfit
from scipy.stats import pearsonr

**The logistic regression models we will compare**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
filename = '/content/cloned-repo/survivalData.csv'

**Loading the data**<br>

Relevant Information: The dataset contains cases from a study that was conducted between 1958 and 1970 at the University of Chicago's Billings Hospital on the survival of patients who had undergone surgery for breast cancer.<br>

Attribute Information:<br>

Age of patient at time of operation (numerical)<br>

Patient's year of operation (year - 1900, numerical)<br>

Number of positive axillary nodes detected (numerical)<br>

Survival status (class attribute)<br>

>1 = the patient survived 5 years or longer<br>

>2 = the patient died within 5 year<br>

In [ ]:
columns = ['age', 'year', 'nodes', 'class']
dataframe = pd.read_csv(filename, header=None, names=columns)

In [ ]:
dataframe.columns

In [ ]:
dataframe['age'].value_counts()

In [ ]:
dataframe['year'].value_counts()

In [ ]:
dataframe['nodes'].value_counts()

In [ ]:
report = dataframe.describe()
print(report)

In [ ]:
dataframe.hist()
plt.show()

**Count the labels for each class**

In [ ]:
target = dataframe['class'].values
counter = Counter(target)
for k,v in counter.items():
  per = v / len(target) * 100
  print('Class=%d, Count=%d, Percentage=%.3f%%' % (k, v, per))

In [ ]:
array = dataframe.values
X = array[:,:3]
Y = array[:,3]

In [ ]:
# Test options and evaluation metric
num_folds = 20
seed = 10
scoring = 'accuracy'

In [ ]:
#create a list of models to evaluate
def get_models():
	models = list()
	models.append(LogisticRegression())
	models.append(RidgeClassifier())
	models.append(SGDClassifier())
	models.append(PassiveAggressiveClassifier())
	models.append(KNeighborsClassifier())
	models.append(DecisionTreeClassifier())
	models.append(ExtraTreeClassifier())
	models.append(LinearSVC())
	models.append(SVC())
	models.append(GaussianNB())
	models.append(AdaBoostClassifier())
	models.append(BaggingClassifier())
	models.append(RandomForestClassifier())
	models.append(ExtraTreesClassifier())
	models.append(GaussianProcessClassifier())
	models.append(GradientBoostingClassifier())
	models.append(LinearDiscriminantAnalysis())
	models.append(QuadraticDiscriminantAnalysis())
	return models

In [ ]:
# evaluate the model using a given test condition
def evaluate_model(cv, model,X,y):
	# evaluate the model
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	# return scores
	return mean(scores)

In [ ]:
# define test conditions
ideal_cv = LeaveOneOut()
cv = KFold(n_splits=10, shuffle=True, random_state=1)
# get the list of models to consider
models = get_models()
# collect results
ideal_results, cv_results = list(), list()
# evaluate each model
for model in models:
	# evaluate model using each test condition
	cv_mean = evaluate_model(cv, model,X,Y)
	ideal_mean = evaluate_model(ideal_cv, model,X,Y)
	# check for invalid results
	if isnan(cv_mean) or isnan(ideal_mean):
		continue
	# store results
	cv_results.append(cv_mean)
	ideal_results.append(ideal_mean)
	# summarize progress
	print('>%s: ideal=%.3f, cv=%.3f' % (type(model).__name__, ideal_mean, cv_mean))

**What methods can we use to try and improve model performance?**

**Binning**<br>
>‘uniform’: All bins in each feature have identical widths.

>‘quantile’: All bins in each feature have the same number of points.

>‘kmeans’: Values in each bin have the same nearest center of a 1D k-means cluster.


In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
X = [[-2, 1, -4,   -1],
     [-1, 2, -3, -0.5],
     [ 0, 3, -2,  0.5],
     [ 1, 4, -1,    2]]
est = KBinsDiscretizer(
    n_bins=3, encode='ordinal', strategy='uniform', subsample=None
)
est.fit(X)
Xt = est.transform(X)
Xt

**Oversampling**<br>
https://imbalanced-learn.org/stable/over_sampling.html


In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import RandomOverSampler


X, y = make_classification(n_samples=5000, n_features=2, n_informative=2,
                           n_redundant=0, n_repeated=0, n_classes=3,
                           n_clusters_per_class=1,
                           weights=[0.01, 0.05, 0.94],
                           class_sep=0.8, random_state=0)

print(sorted(Counter(y).items()))

In [ ]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))